# simulation of single-cell count data

In [2]:
# Load package
library(scater)
library(splatter)
library(scuttle)

Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    

In [3]:
real_rwa_counts <- read.csv('rawCounts_10X_endothelium.csv',row.names=1)
real_rwa_counts <-t(real_rwa_counts)

In [4]:
dim(real_rwa_counts)
real_rwa_counts[1:5,1:5]

[1] 19050   158

,AAACGGGCACTCGACG-1,AAACGGGTCAACACCA-1,AAAGTAGTCCTCCTAG-1,AACCGCGAGAAGATTC-1,AACCGCGTCTCGTTTA-1
RP11.34P13.7,0,0,0,0,0
FO538757.2,0,1,1,0,0
AP006222.2,0,4,0,3,0
RP4.669L17.2,0,0,0,0,0
RP4.669L17.10,0,0,0,0,0


In [5]:
params <-splatEstimate(real_rwa_counts)

In [6]:
# Set multiple parameters at once (using a list)
params <- setParams(params, update = list(nGenes = 20000, batchCells=100,
                                          mean.shape=0.358,mean.rate=2.734,
                                          lib.loc=9.034,lib.scale=0.435,
                                          out.prob=0.006,out.facLoc=6.420,out.facScale=0.653,
                                          bcv.common = 0.220,bcv.df = 21.714,
                                          dropout.mid=-0.237,dropout.shape=-1.115))

# 1. different cell numbers 

In [7]:
nCell=c(50,100,500,1000,2500,5000);
params <- newSplatParams();
for (i in 1:6){
    print(paste('simulating the cell number: ',
                as.character(nCell[i])
               )
         );
    params <- setParams(params, update = list(nGenes = 10000, 
                                              batchCells=nCell[i],
                                              group.prob = c(0.5, 0.5),
                                              de.prob = c(0.05,0.05),
                                              de.facLoc = c(-0.1, -0.1),
                                              dropout.type='experiment',
                                              de.facScale = c(0.3,0.3)
                                             )
                       );
    sim.G2 <- splatSimulate(params,method = "groups",
                            verbose = FALSE);
    sim.G2 <- addGeneLengths(sim.G2);
    tpm(sim.G2) <- calculateTPM(sim.G2, rowData(sim.G2)$Length);
    sim.G2 <- logNormCounts(sim.G2, log=FALSE);
    sim.G2 <- logNormCounts(sim.G2);
    saveRDS(sim.G2,
            file=paste('./write/SplatSim_G2_B1_sce_based_real_data_nCell_',
                       as.character(nCell[i]),
                       '_20210903.rds',
                       sep='')
           );
    write.csv(rowData(sim.G2),
              file=paste('./write/SplatSim_G2_B1_sce_based_real_data_feature_nCell_',
                         as.character(nCell[i]),
                         '_20210903.csv',
                         sep='')
             );
    write.csv(colData(sim.G2),
              file=paste('./write/SplatSim_G2_B1_sce_based_real_data_pheno_nCell_',
                         as.character(nCell[i]),
                         '_20210903.csv',
                         sep='')
             );
    write.csv(counts(sim.G2),
              file=paste('./write/SplatSim_G2_B1_sce_based_real_data_counts_nCell_',
                         as.character(nCell[i]),
                         '_20210903.csv',
                         sep='')
             );
}

[1] "simulating the cell number:  50"


Warning message in sparsifyMatrices(assays(sim), auto = TRUE, verbose = verbose):
“matrix 'Dropout' is class 'matrixarray', unable to estimate size reduction factor”


[1] "simulating the cell number:  100"


Warning message in sparsifyMatrices(assays(sim), auto = TRUE, verbose = verbose):
“matrix 'Dropout' is class 'matrixarray', unable to estimate size reduction factor”


[1] "simulating the cell number:  500"


...

Warning message in sparsifyMatrices(assays(sim), auto = TRUE, verbose = verbose):
“matrix 'Dropout' is class 'matrixarray', unable to estimate size reduction factor”


[1] "simulating the cell number:  2500"


Warning message in sparsifyMatrices(assays(sim), auto = TRUE, verbose = verbose):
“matrix 'Dropout' is class 'matrixarray', unable to estimate size reduction factor”


[1] "simulating the cell number:  5000"


Warning message in sparsifyMatrices(assays(sim), auto = TRUE, verbose = verbose):
“matrix 'Dropout' is class 'matrixarray', unable to estimate size reduction factor”


# 2. different number of cell groups

In [18]:
nGroup <- c(2,5,10,25,50);
for (i in 1:5){
    G <- nGroup[i];
    n_cell = 200*G;
    #x1=2/((G+1)*G);
    #x2= 2/(G+1);
    #x3 = 2/((G+1)*G);
    #g_p<-seq(from = x1,to = x2,by=x3)
    params <- newSplatParams();
    print(paste('simulating the cell group: ',
                as.character(G)
               )
         );
    params <- setParams(params, update = list(nGenes = 10000, 
                                              batchCells=n_cell,
                                              group.prob = rep(1/G,G),
                                              de.prob = rep(0.05,G),
                                              de.facLoc = rep(-0.1, G),
                                              dropout.type='experiment',
                                              de.facScale = rep(0.3,G)
                                             )
                       );
    sim.G2 <- splatSimulate(params,method = "groups",
                            verbose = FALSE);
    sim.G2 <- addGeneLengths(sim.G2);
    tpm(sim.G2) <- calculateTPM(sim.G2, rowData(sim.G2)$Length);
    sim.G2 <- logNormCounts(sim.G2, log=FALSE);
    sim.G2 <- logNormCounts(sim.G2);
    saveRDS(sim.G2,
            file=paste('./write/SplatSim_eC200_B1_sce_de_prob_nGroup_',
                       as.character(G),
                       '_20210903.rds',
                       sep='')
           );
    write.csv(rowData(sim.G2),
              file=paste('./write/SplatSim_eC200_B1_feature_de_prob_nGroup_',
                         as.character(G),
                         '_20210903.csv',
                         sep='')
             );
    write.csv(colData(sim.G2),
              file=paste('./write/SplatSim_eC200_B1_pheno_de_prob_nGroup_',
                         as.character(G),
                         '_20210903.csv',
                         sep='')
             );
    write.csv(counts(sim.G2),
              file=paste('./write/SplatSim_eC200_B1_counts_de_prob_nGroup_',
                         as.character(G),
                         '_20210903.csv',
                         sep='')
             );
}

[1] "simulating the cell group:  2"


Warning message in sparsifyMatrices(assays(sim), auto = TRUE, verbose = verbose):
“matrix 'Dropout' is class 'matrixarray', unable to estimate size reduction factor”


[1] "simulating the cell group:  5"


Warning message in sparsifyMatrices(assays(sim), auto = TRUE, verbose = verbose):
“matrix 'Dropout' is class 'matrixarray', unable to estimate size reduction factor”


[1] "simulating the cell group:  10"


Warning message in sparsifyMatrices(assays(sim), auto = TRUE, verbose = verbose):
“matrix 'Dropout' is class 'matrixarray', unable to estimate size reduction factor”


[1] "simulating the cell group:  25"


Warning message in sparsifyMatrices(assays(sim), auto = TRUE, verbose = verbose):
“matrix 'Dropout' is class 'matrixarray', unable to estimate size reduction factor”


[1] "simulating the cell group:  50"


Warning message in sparsifyMatrices(assays(sim), auto = TRUE, verbose = verbose):
“matrix 'Dropout' is class 'matrixarray', unable to estimate size reduction factor”


In [7]:
nGroup <- c(2,5,10,25,50);
n_cell = 10000
for (i in 1:5){
    G <- nGroup[i];
    params <- newSplatParams();
    print(paste('simulating the cell group: ',
                as.character(G)
               )
         );
    params <- setParams(params, update = list(nGenes = 10000, 
                                              batchCells=n_cell,
                                              group.prob = rep(1/G,G),
                                              de.prob = rep(0.05,G),
                                              de.facLoc = rep(-0.1, G),
                                              dropout.type='experiment',
                                              de.facScale = rep(0.3,G)
                                             )
                       );
    sim.G2 <- splatSimulate(params,method = "groups",
                            verbose = FALSE);
    sim.G2 <- addGeneLengths(sim.G2);
    tpm(sim.G2) <- calculateTPM(sim.G2, rowData(sim.G2)$Length);
    sim.G2 <- logNormCounts(sim.G2, log=FALSE);
    sim.G2 <- logNormCounts(sim.G2);
    saveRDS(sim.G2,
            file=paste('./write/SplatSim_nC10k_B1_sce_de_prob_nGroup_',
                       as.character(G),
                       '_20210903.rds',
                       sep='')
           );
    write.csv(rowData(sim.G2),
              file=paste('./write/SplatSim_nC10k_B1_feature_de_prob_nGroup_',
                         as.character(G),
                         '_20210903.csv',
                         sep='')
             );
    write.csv(colData(sim.G2),
              file=paste('./write/SplatSim_nC10k_B1_pheno_de_prob_nGroup_',
                         as.character(G),
                         '_20210903.csv',
                         sep='')
             );
    write.csv(counts(sim.G2),
              file=paste('./write/SplatSim_nC10k_B1_counts_de_prob_nGroup_',
                         as.character(G),
                         '_20210903.csv',
                         sep='')
             );
}

[1] "simulating the cell group:  2"


Warning message in sparsifyMatrices(assays(sim), auto = TRUE, verbose = verbose):
“matrix 'Dropout' is class 'matrixarray', unable to estimate size reduction factor”


[1] "simulating the cell group:  5"


Warning message in sparsifyMatrices(assays(sim), auto = TRUE, verbose = verbose):
“matrix 'Dropout' is class 'matrixarray', unable to estimate size reduction factor”


[1] "simulating the cell group:  10"


Warning message in sparsifyMatrices(assays(sim), auto = TRUE, verbose = verbose):
“matrix 'Dropout' is class 'matrixarray', unable to estimate size reduction factor”


[1] "simulating the cell group:  25"


Warning message in sparsifyMatrices(assays(sim), auto = TRUE, verbose = verbose):
“matrix 'Dropout' is class 'matrixarray', unable to estimate size reduction factor”


[1] "simulating the cell group:  50"


Warning message in sparsifyMatrices(assays(sim), auto = TRUE, verbose = verbose):
“matrix 'Dropout' is class 'matrixarray', unable to estimate size reduction factor”


# 3. different cell number in each group

In [56]:
G <- 10;
n_cell = 3000;
r_dist = c('uniform','norm','gamma','beta','poisson');
for (i in 1:5){
    if (r_dist[i]=='uniform'){
        g_p = rep(1/G,G);
    } 
    else if (r_dist[i]=='norm'){
        x1 = abs(rnorm(10));
        g_p = x1/sum(x1);
    } 
    else if (r_dist[i]=='gamma'){
        x1 = abs(rgamma(10,1));
        g_p = x1/sum(x1);
    } 
    else if (r_dist[i]=='beta') {
        x1 = abs(rbeta(10,1,2));
        g_p = x1/sum(x1);
    } 
    else {
        x1 = abs(rpois(10,1));
        g_p = x1/sum(x1);
    }
    params <- newSplatParams();
    print(paste('simulating the distribution: ',
                r_dist[i]
               )
         );

    params <- setParams(params, update = list(nGenes = 10000, 
                                              batchCells=n_cell,
                                              group.prob = g_p,
                                              de.prob = rep(0.05,G),
                                              de.facLoc = rep(-0.1, G),
                                              dropout.type='experiment',
                                              de.facScale = rep(0.3,G)
                                             )
                       );
    sim.G2 <- splatSimulate(params,method = "groups",
                            verbose = FALSE);
    sim.G2 <- addGeneLengths(sim.G2);
    tpm(sim.G2) <- calculateTPM(sim.G2, rowData(sim.G2)$Length);
    sim.G2 <- logNormCounts(sim.G2, log=FALSE);
    sim.G2 <- logNormCounts(sim.G2);
    saveRDS(sim.G2,
            file=paste('./write/SplatSim_G10_nC3k_B1_sce_de_prob_',
                       r_dist[i],
                       '_20210903.rds',
                       sep='')
           );
    write.csv(rowData(sim.G2),
              file=paste('./write/SplatSim_G10_nC3k_B1_feature_sce_de_prob_',
                         r_dist[i],
                         '_20210903.csv',
                         sep='')
             );
    write.csv(colData(sim.G2),
              file=paste('./write/SplatSim_G10_nC3k_B1_pheno_sce_de_prob_',
                         r_dist[i],
                         '_20210903.csv',
                         sep='')
             );
    write.csv(counts(sim.G2),
              file=paste('./write/SplatSim_G10_nC3k_B1_counts_sce_de_prob_',
                         r_dist[i],
                         '_20210903.csv',
                         sep='')
             );
}

[1] "simulating the distribution:  uniform"


Warning message in sparsifyMatrices(assays(sim), auto = TRUE, verbose = verbose):
“matrix 'Dropout' is class 'matrixarray', unable to estimate size reduction factor”


[1] "simulating the distribution:  norm"


Warning message in sparsifyMatrices(assays(sim), auto = TRUE, verbose = verbose):
“matrix 'Dropout' is class 'matrixarray', unable to estimate size reduction factor”


[1] "simulating the distribution:  gamma"


Warning message in sparsifyMatrices(assays(sim), auto = TRUE, verbose = verbose):
“matrix 'Dropout' is class 'matrixarray', unable to estimate size reduction factor”


[1] "simulating the distribution:  beta"


Warning message in sparsifyMatrices(assays(sim), auto = TRUE, verbose = verbose):
“matrix 'Dropout' is class 'matrixarray', unable to estimate size reduction factor”


[1] "simulating the distribution:  poisson"


Warning message in sparsifyMatrices(assays(sim), auto = TRUE, verbose = verbose):
“matrix 'Dropout' is class 'matrixarray', unable to estimate size reduction factor”


# 4. different Batches

In [57]:
G <- 10;
n_cell = 3000;
#x1=2/((G+1)*G);
#x2= 2/(G+1);
#x3 = 2/((G+1)*G);
#g_p<-seq(from = x1,to = x2,by=x3);
batch_n= c(1,2,4,6,8,10);
for (i in 1:6){
    B <- batch_n[i];
    params <- newSplatParams();
    print(paste('simulating the batch: ',
                as.character(B)
               )
         );
    params <- setParams(params, update = list(nGenes = 10000, 
                                              batchCells=rep(round(n_cell/B),B),
                                              group.prob = rep(1/G,G),
                                              de.prob = rep(0.05,G),
                                              de.facLoc = rep(-0.1, G),
                                              dropout.type='experiment',
                                              de.facScale = rep(0.3,G)
                                             )
                       );
    sim.G2 <- splatSimulate(params,method = "groups",
                            verbose = FALSE);
    sim.G2 <- addGeneLengths(sim.G2);
    tpm(sim.G2) <- calculateTPM(sim.G2, rowData(sim.G2)$Length);
    sim.G2 <- logNormCounts(sim.G2, log=FALSE);
    sim.G2 <- logNormCounts(sim.G2);
    saveRDS(sim.G2,
            file=paste('./write/SplatSim_G10_nC3k_sce_de_prob_nBatch_',
                       as.character(B),
                       '_20210903.rds',
                       sep='')
           );
    write.csv(rowData(sim.G2),
              file=paste('./write/SplatSim_G10_nC3k_feature_de_prob_nBatch_',
                         as.character(B),
                         '_20210903.csv',
                         sep='')
             );
    write.csv(colData(sim.G2),
              file=paste('./write/SplatSim_G10_nC3k_pheno_de_prob_nBatch_',
                         as.character(B),
                         '_20210903.csv',
                         sep='')
             );
    write.csv(counts(sim.G2),
              file=paste('./write/SplatSim_G10_nC3k_counts_de_prob_nBatch_',
                         as.character(B),
                         '_20210903.csv',
                         sep='')
             );
}

[1] "simulating the batch:  1"


Warning message in sparsifyMatrices(assays(sim), auto = TRUE, verbose = verbose):
“matrix 'Dropout' is class 'matrixarray', unable to estimate size reduction factor”


[1] "simulating the batch:  2"


Warning message in sparsifyMatrices(assays(sim), auto = TRUE, verbose = verbose):
“matrix 'Dropout' is class 'matrixarray', unable to estimate size reduction factor”


[1] "simulating the batch:  4"


...

Warning message in sparsifyMatrices(assays(sim), auto = TRUE, verbose = verbose):
“matrix 'Dropout' is class 'matrixarray', unable to estimate size reduction factor”


[1] "simulating the batch:  8"


Warning message in sparsifyMatrices(assays(sim), auto = TRUE, verbose = verbose):
“matrix 'Dropout' is class 'matrixarray', unable to estimate size reduction factor”


[1] "simulating the batch:  10"


Warning message in sparsifyMatrices(assays(sim), auto = TRUE, verbose = verbose):
“matrix 'Dropout' is class 'matrixarray', unable to estimate size reduction factor”


In [58]:
sim.G2

class: SingleCellExperiment 
dim: 10000 3000 
metadata(1): Params
assays(11): BatchCellMeans BaseCellMeans ... normcounts logcounts
rownames(10000): Gene1 Gene2 ... Gene9999 Gene10000
rowData names(25): Gene BaseGeneMean ... DEFacGroup10 Length
colnames(3000): Cell1 Cell2 ... Cell2999 Cell3000
colData names(5): Cell Batch Group ExpLibSize sizeFactor
reducedDimNames(0):
altExpNames(0):

In [59]:
colData(sim.G2)

DataFrame with 3000 rows and 5 columns
                Cell       Batch    Group ExpLibSize sizeFactor
         <character> <character> <factor>  <numeric>  <numeric>
Cell1          Cell1      Batch1   Group6    44739.2   0.716300
Cell2          Cell2      Batch1   Group8    80208.5   1.328167
Cell3          Cell3      Batch1   Group8    41733.6   0.676401
Cell4          Cell4      Batch1   Group8    70017.7   1.169051
Cell5          Cell5      Batch1   Group8    62497.5   1.038066
...              ...         ...      ...        ...        ...
Cell2996    Cell2996     Batch10   Group9    54533.4   0.875630
Cell2997    Cell2997     Batch10   Group8    57234.3   0.946145
Cell2998    Cell2998     Batch10   Group7    58286.0   0.951363
Cell2999    Cell2999     Batch10   Group3    60910.2   0.996924
Cell3000    Cell3000     Batch10   Group5    55357.2   0.904223